In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [2]:
with open('dracula.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
77


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_text = encode(text[:200])
decoded_text = decode(encoded_text)
print(encoded_text)
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

[27, 41, 24, 26, 44, 35, 24, 0, 0, 52, 75, 0, 0, 25, 68, 51, 63, 1, 42, 70, 65, 61, 55, 68, 0, 0, 0, 12, 19, 20, 18, 1, 55, 54, 59, 70, 59, 65, 64, 0, 0, 0, 0, 0, 43, 24, 25, 35, 28, 1, 38, 29, 1, 26, 38, 37, 43, 28, 37, 43, 42, 0, 0, 0, 26, 31, 24, 39, 43, 28, 41, 0, 0, 1, 1, 12, 1, 1, 33, 65, 64, 51, 70, 58, 51, 64, 1, 31, 51, 68, 61, 55, 68, 5, 69, 1, 33, 65, 71, 68, 64, 51, 62, 0, 1, 1, 13, 1, 1, 33, 65, 64, 51, 70, 58, 51, 64, 1, 31, 51, 68, 61, 55, 68, 5, 69, 1, 33, 65, 71, 68, 64, 51, 62, 0, 1, 1, 14, 1, 1, 33, 65, 64, 51, 70, 58, 51, 64, 1, 31, 51, 68, 61, 55, 68, 5, 69, 1, 33, 65, 71, 68, 64, 51, 62, 0, 1, 1, 15, 1, 1, 33, 65, 64, 51, 70, 58, 51, 64, 1, 31, 51, 68, 61, 55, 68, 5, 69, 1, 33, 65, 71, 68, 64, 51, 62, 0, 1, 1, 16]
tensor([27, 41, 24, 26, 44, 35, 24,  0,  0, 52, 75,  0,  0, 25, 68, 51, 63,  1,
        42, 70, 65, 61, 55, 68,  0,  0,  0, 12, 19, 20, 18,  1, 55, 54, 59, 70,
        59, 65, 64,  0,  0,  0,  0,  0, 43, 24, 25, 35, 28,  1, 38, 29,  1, 26,
        38, 37

In [4]:
n = int(0.8*len(text))
train = data[:n]
val = data[n:]
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iter = 250
dropout = 0.2

def get_batch(split):
    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return(x,y)

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[64, 65, 70,  1, 59, 70, 10,  1],
        [63, 69,  8,  1, 56, 68, 65, 63],
        [54,  1, 54, 51, 63, 66, 62, 75],
        [68,  1, 73, 55, 68, 55,  1, 64]])
targets:
tensor([[65, 70,  1, 59, 70, 10,  1,  1],
        [69,  8,  1, 56, 68, 65, 63,  1],
        [ 1, 54, 51, 63, 66, 62, 75,  1],
        [ 1, 73, 55, 68, 55,  1, 64, 65]])


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


w.w-bV(g u",tLsF-:3mrRnp4QmMEEFq&pSK4l3ttjOR1ioD(-
wCDFgoq-
fVH7nskZG)V1QYm)Y.kfd'K&h qsPP0b
D2ys6'Lci83L'd"_3_s
en0ee)_EXa-lu
OVZbTu"BMsFg0Iwf'1D_iLHFeG8;mb GOV1iErM2GCtPjo1mK3x7;N;,ViL"SEPRgfRR_:whV(-!H?rq?PQrOVsHH;-i?POS""2MzfX?N4BU4'jcH 
XQjgYyKjwMWD_uQ.8o
!mRd7Y6jOW3G:!xo!(UIm4:tLJa?&X1sk?CxvMIV1!z6zByadgYoO"kqb.AKAR5DxfT:uVHIwWjCdRFyaVXz':)uWjO0!Y I
d"MdiY"Te7v3RAO?Za.fFrR 
4 dR5DnI,w8j(ne.BpdKDQK;zVe5suU&2RwKGmhxf7Ksp,XQyMdgUKP!qXQUU9ixGC6&aPBNTKoC68!WVqJvkYbGTW'e5.AYyd,3.Z2ka-gxuJd""l1gf


In [1]:
# create a PyTourch optimiser

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iter ==0:
        losses = estimate_loss()
        print(f"step:{iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    #sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

NameError: name 'torch' is not defined

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=1000)[0].tolist())
print(generated_chars)

In [ ]:
# Mean Squared Error (MSE): MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
# Gradient Descent (GD): is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
# Momentum: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
# RMSprop: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
# Adam: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
# AdamW: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.